In [1]:
import pandas as pd
import numpy as np

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/sample_data/train.csv')

# Display the DataFrame
print(df)


      Unnamed: 0                              Name    Location  Year  \
0              1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1              2                      Honda Jazz V     Chennai  2011   
2              3                 Maruti Ertiga VDI     Chennai  2012   
3              4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4              6            Nissan Micra Diesel XV      Jaipur  2013   
...          ...                               ...         ...   ...   
5842        6014                  Maruti Swift VDI       Delhi  2014   
5843        6015          Hyundai Xcent 1.1 CRDi S      Jaipur  2015   
5844        6016             Mahindra Xylo D4 BSIV      Jaipur  2012   
5845        6017                Maruti Wagon R VXI     Kolkata  2013   
5846        6018             Chevrolet Beat Diesel   Hyderabad  2011   

      Kilometers_Driven Fuel_Type Transmission Owner_Type     Mileage  \
0                 41000    Diesel       Manual      First  19.

In [2]:
rows, columns = df.shape

print(f"Number of rows: {rows}")
print(f"Number of columns: {columns}")

Number of rows: 5847
Number of columns: 14


In [3]:
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In this code, I converted string values in the Power column to numeric values. By setting errors='coerce', any non-convertible values are transformed into NaN.

To fill in missing values, I used the median instead of the mean or mode because the Car power ratings can vary a lot, and using the mean could be misleading because each car has different value. The median is a better choice here because it’s less affected by these extremes and gives a more accurate central value. While the mode shows the most common value.

In [12]:
# Clean and convert 'Power' column
df['Power'] = df['Power'].astype(str).str.replace(' bhp', '')  # Convert to string and remove ' bhp'
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')       # Convert the values to numeric and replace the values that cannot be converted to a number to NaN
df['Power'].fillna(df['Power'].median())          # Impute missing values with the median


,Power
0,126.20
1,88.70
2,88.76
3,140.80
4,63.10
...,...
5842,74.00
5843,71.00
5844,112.00
5845,67.10


Since the Mileage column is similar to the Power column , I used the median to impute missing values

In [13]:
# Clean and convert 'Mileage' column
df['Mileage'] = df['Mileage'].astype(str).str.replace(' kmpl', '').str.replace(' km/kg', '')  # Convert to string first
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')    # Convert to numeric value  with errors converted to NaN
df['Mileage'].fillna(df['Mileage'].median())       # Impute missing values with the median

,Mileage
0,19.67
1,13.00
2,20.77
3,15.20
4,23.08
...,...
5842,28.40
5843,24.40
5844,14.00
5845,18.90


Since the engine column is similar to the Power column , I used the median to impute missing values

In [6]:
# Clean and convert 'Engine' column
df['Engine'] = df['Engine'].astype(str).str.replace(' CC', '') # Convert to string first
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce') # Convert to numeric value  with errors converted to NaN
df['Engine'].fillna(df['Engine'].median()) # Impute missing values with the median

,Engine
0,1582.0
1,1199.0
2,1248.0
3,1968.0
4,1461.0
...,...
5842,1248.0
5843,1120.0
5844,2498.0
5845,998.0


I used the mode to impute missing values in the Seats column because seating configurations are typically standard across cars, such as five-seaters or six-seaters.

In [7]:
# Fill missing values in 'Seats' with mode
df['Seats'].fillna(df['Seats'].mode()[0])


,Seats
0,5.0
1,5.0
2,7.0
3,5.0
4,5.0
...,...
5842,5.0
5843,5.0
5844,8.0
5845,5.0


In [8]:
categorical_cols = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Location']

data = pd.get_dummies(df, columns=categorical_cols, drop_first=True) #convert the values in to dummies and drop first column

print(data.head())

   Unnamed: 0                              Name  Year  Kilometers_Driven  \
0           1  Hyundai Creta 1.6 CRDi SX Option  2015              41000   
1           2                      Honda Jazz V  2011              46000   
2           3                 Maruti Ertiga VDI  2012              87000   
3           4   Audi A4 New 2.0 TDI Multitronic  2013              40670   
4           6            Nissan Micra Diesel XV  2013              86999   

   Mileage  Engine   Power  Seats  New_Price  Price  ...  Location_Bangalore  \
0    19.67  1582.0  126.20    5.0        NaN  12.50  ...               False   
1    13.00  1199.0   88.70    5.0  8.61 Lakh   4.50  ...               False   
2    20.77  1248.0   88.76    7.0        NaN   6.00  ...               False   
3    15.20  1968.0  140.80    5.0        NaN  17.74  ...               False   
4    23.08  1461.0   63.10    5.0        NaN   3.50  ...               False   

   Location_Chennai  Location_Coimbatore  Location_Delhi  Loca

In [14]:
# Calculate the Mileage to Engine Size Ratio for  better fuel-efficient engines.
data['Mileage_to_Engine'] = data['Mileage'] / data['Engine']

# Display the new feature
print(data[['Name', 'Mileage', 'Engine', 'Mileage_to_Engine']].head())

                               Name  Mileage  Engine  Mileage_to_Engine
0  Hyundai Creta 1.6 CRDi SX Option    19.67  1582.0           0.012434
1                      Honda Jazz V    13.00  1199.0           0.010842
2                 Maruti Ertiga VDI    20.77  1248.0           0.016643
3   Audi A4 New 2.0 TDI Multitronic    15.20  1968.0           0.007724
4            Nissan Micra Diesel XV    23.08  1461.0           0.015797


In [25]:

selected_data = data[['Name', 'Year', 'Price']]

# Filter rows where mileage above 15
filtered_data = data[data['Mileage'] > 15]

# Rename columns
renamed_data = data.rename(columns={'fuel_type': 'fuel_transmission', 'Engine': 'Engine_capacity'})

# Using mutate operation to create a new column
df['Mileage_Per_Power'] = df['Mileage'] / df['Power']

# Sort by Milage
arranged_data = data.sort_values(by='Mileage')

# Summarize data: calculate average and max price by location
summary_data = data.groupby('Price').agg(
    avg_price=('Price', 'mean'),
    max_price=('Price', 'max')
).reset_index()

